# Guía de Uso - Detector de Vulnerabilidades con CodeBERT

## Contenido

1. **[Uso Directo (Modelo Pre-entrenado)](#1-uso-directo)** - Carga el modelo y empieza a predecir
2. **[Modelo Pre-cargado (Drive)](#2-modelo-pre-cargado)** - Descarga el modelo entrenado por 5 épocas
3. **[Entrenamiento Propio](#3-entrenamiento-propio)** - Entrena tu propio modelo desde cero

---

## Clases de Vulnerabilidades Detectadas

| ID | Clase | Descripción |
|----|-------|-------------|
| 0 | `Safe` | Código seguro |
| 1 | `CWE-79` | Cross-Site Scripting (XSS) |
| 2 | `CWE-89` | SQL Injection |
| 3 | `CWE-78` | OS Command Injection |
| 4 | `CWE-22` | Path Traversal |
| 5 | `CWE-434` | Unrestricted File Upload |
| 6 | `CWE-352` | Cross-Site Request Forgery (CSRF) |
| 7 | `Other` | Otras vulnerabilidades |

# 1. Uso Directo (Modelo Pre-entrenado)

Esta sección te permite cargar un modelo ya entrenado y empezar a hacer predicciones inmediatamente.

## 1.1 Instalar Dependencias



In [ ]:
# Instalar dependencias (solo ejecutar una vez)
%pip install torch transformers --quiet

## 1.2 Cargar el Predictor

El `VulnerabilityPredictor` encapsula toda la lógica de tokenización, sliding window y predicción.

In [ ]:
import sys
import os

# Agregar el directorio raíz al path
sys.path.append(os.path.abspath('..'))

from src.v3.predictor import VulnerabilityPredictor

# Configuración
MODEL_PATH = "../models/codebert_vuln/best_model.bin"
MODEL_NAME = "microsoft/codebert-base"

# Cargar el predictor
print("🔄 Cargando modelo CodeBERT...")
predictor = VulnerabilityPredictor(
    model_path=MODEL_PATH,
    model_name=MODEL_NAME
)
print("✅ Modelo cargado exitosamente!")

## 1.3 Hacer Predicciones

### Ejemplo 1: SQL Injection

In [ ]:
# Código vulnerable: SQL Injection
sql_injection_code = '''
def get_user(username):
    query = "SELECT * FROM users WHERE username = '" + username + "'"
    cursor.execute(query)
    return cursor.fetchone()
'''

result = predictor.predict(sql_injection_code)

print("🔍 Análisis de SQL Injection:")
print(f"   Predicción: {result['label']}")
print(f"   Confianza: {result['confidence']:.2%}")
print(f"\n📊 Probabilidades:")
for label, prob in sorted(result['probabilities'].items(), key=lambda x: -x[1])[:3]:
    print(f"   {label}: {prob:.2%}")

### Ejemplo 2: Cross-Site Scripting (XSS)

In [ ]:
# Código vulnerable: XSS
xss_code = '''
@app.route('/search')
def search():
    query = request.args.get('q')
    return f"<h1>Results for: {query}</h1>"
'''

result = predictor.predict(xss_code)

print("🔍 Análisis de XSS:")
print(f"   Predicción: {result['label']}")
print(f"   Confianza: {result['confidence']:.2%}")

### Ejemplo 3: Código Seguro

In [ ]:
# Código seguro: Consulta parametrizada
safe_code = '''
def get_user_secure(username):
    query = "SELECT * FROM users WHERE username = %s"
    cursor.execute(query, (username,))
    return cursor.fetchone()
'''

result = predictor.predict(safe_code)

print("🔍 Análisis de Código Seguro:")
print(f"   Predicción: {result['label']}")
print(f"   Confianza: {result['confidence']:.2%}")

### Ejemplo 4: Analiza tu propio código

Modifica la variable `mi_codigo` con el código que quieras analizar:

In [ ]:
# 📝 Pega tu código aquí
mi_codigo = '''
# Tu código aquí...
def mi_funcion():
    pass
'''

# Analizar
result = predictor.predict(mi_codigo)

print("=" * 50)
print("🔍 RESULTADO DEL ANÁLISIS")
print("=" * 50)
print(f"📌 Predicción: {result['label']}")
print(f"📊 Confianza: {result['confidence']:.2%}")
print(f"\n📈 Top 3 Probabilidades:")
for label, prob in sorted(result['probabilities'].items(), key=lambda x: -x[1])[:3]:
    bar = "█" * int(prob * 20)
    print(f"   {label:12} {bar} {prob:.2%}")

---

# 2. Modelo Pre-cargado (Google Drive)

Si no quieres entrenar el modelo desde cero, puedes descargar el modelo pre-entrenado con **5 épocas**.

## 🔗 Link de Descarga

| Archivo | Épocas | F1-Score | Link |
|---------|--------|----------|------|
| `best_model.bin` | 5 | ~0.75 | [📥 Descargar desde Google Drive](https://drive.google.com/file/d/1s7ojteSg1E7X_rzMVCKCJKil44sZ0-YO/view?usp=sharing) |

## 📁 Instrucciones de Instalación

1. **Descargar** el archivo `best_model.bin` del link de arriba
2. **Colocar** el archivo en la carpeta `models/codebert_vuln/`
3. **Verificar** la estructura:

```
Lab_Mineria_de_Datos/
├── models/
│   └── codebert_vuln/
│       └── best_model.bin    <-- Aquí debe estar
├── notebooks/
│   └── 00_guide.ipynb        <-- Este archivo
└── src/
    └── v3/
        └── predictor.py
```

4. **Ejecutar** las celdas de la sección anterior para cargar y usar el modelo

> ⚠️ **Nota**: El modelo pesa aproximadamente ~500MB. Asegúrate de tener suficiente espacio en disco.

---

# 3. Entrenamiento Propio

Si deseas entrenar tu propio modelo con tus datos o mejorar el existente, sigue estos pasos.

## 3.1 Requisitos Previos

### Hardware Recomendado
- **GPU**: NVIDIA con al menos 8GB VRAM (RTX 3060 o superior)
- **RAM**: 16GB mínimo
- **Disco**: 10GB libres

### Dataset
El dataset debe estar en `data/processed/dataset_ml_ready.csv` con las columnas:
- `code`: Código fuente de la función
- `cwe_id`: Etiqueta de vulnerabilidad (ej: "CWE-79", "Safe")

## 3.2 Paso 1: Pre-procesar el Dataset

El pre-procesamiento tokeniza el código y crea ventanas deslizantes para manejar código largo.

> **Importante**: Este paso hace el split train/val para evitar **data leakage**

In [ ]:
# Ejecutar desde la terminal (toma varios minutos)
# !python pre_process.py

# O ejecutar directamente:
import subprocess
result = subprocess.run(['python', 'pre_process.py'], capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print(f"Error: {result.stderr}")

Después de ejecutar, deberías tener:
```
notebooks/
├── train_cache/    # Datos de entrenamiento (40 partes)
│   ├── part_0.pt
│   ├── part_1.pt
│   └── ...
└── val_cache/      # Datos de validación (40 partes)
    ├── part_0.pt
    └── ...
```

## 3.3 Paso 2: Entrenar el Modelo

Abre el notebook `04_codebert_ft.ipynb` y ejecuta las celdas en orden.

### Hiperparámetros Configurables

```python
# Épocas y batch
EPOCHS = 7                  # Número de épocas
BATCH_SIZE = 8              # Batch size (reducir si hay OOM)
ACCUMULATION_STEPS = 4      # Batch efectivo = 8 * 4 = 32

# Learning rate y regularización
LEARNING_RATE = 2e-5        # Óptimo para fine-tuning
WEIGHT_DECAY = 0.01         # Regularización L2
WARMUP_RATIO = 0.1          # 10% warmup

# Focal Loss (manejo de desbalance)
FOCAL_GAMMA = 2.0           # Mayor = más enfoque en ejemplos difíciles

# Sliding Window (código largo)
MAX_LEN = 512               # Tokens por ventana
STRIDE = 256                # 50% overlap
MAX_WINDOWS = 8             # Máximo de ventanas

# Early Stopping
PATIENCE = 3                # Épocas sin mejora antes de parar
```

## 3.4 Paso 3: Usar el Modelo Entrenado

Una vez entrenado, el modelo se guarda en `models/codebert_vuln/best_model.bin`.

Regresa a la **Sección 1** de esta guía para cargar y usar tu modelo.

---

# 4. API REST (Opcional)

También puedes usar el modelo a través de una API REST para integración en CI/CD.

## Iniciar el servidor

In [ ]:
# Ejecutar en una terminal separada:
# cd src && python analysis_api.py

# El servidor estará disponible en http://localhost:8000
print("Para iniciar el servidor API, ejecuta en una terminal:")
print("  cd src && python analysis_api.py")
print("\nEndpoints disponibles:")
print("  GET  /          - Health check")
print("  POST /predict   - Predicción directa de código")
print("  POST /analyze   - Análisis completo de archivos")

## Ejemplo de uso con cURL

```bash
# Predicción directa
curl -X POST "http://localhost:8000/predict" \
  -H "Content-Type: application/json" \
  -d '{"code": "def login(u,p): cursor.execute(\"SELECT * FROM users WHERE user=\" + u)"}'
```

## Ejemplo de uso con Python requests

In [ ]:
import requests

# URL del servidor (asegúrate de que esté corriendo)
API_URL = "http://localhost:8000"

# Ejemplo de predicción
codigo_a_analizar = '''
def procesar_archivo(filename):
    path = "/var/data/" + filename
    with open(path, "r") as f:
        return f.read()
'''

try:
    response = requests.post(
        f"{API_URL}/predict",
        json={"code": codigo_a_analizar}
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"Predicción: {result['label']}")
        print(f"Confianza: {result['confidence']:.2%}")
    else:
        print(f"Error: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("No se pudo conectar al servidor. Asegúrate de que esté corriendo.")